[Build an agent](https://python.langchain.com/docs/tutorials/agents/)

# End to end agent

In [2]:
# Import relevant functionality
# from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
# model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
model = ChatOpenAI(model_name="gpt-4o-mini")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [3]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in madrid, spain")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in madrid, spain
================================== Ai Message ==================================

Hello Bob! It's great to hear from someone in Madrid. How can I assist you today?


In [4]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_j1pK7p07NOeiV5Bpa8tPHqa0)
 Call ID: call_j1pK7p07NOeiV5Bpa8tPHqa0
  Args:
    query: current weather in Madrid, Spain
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Madrid', 'region': 'Madrid', 'country': 'Spain', 'lat': 40.4, 'lon': -3.6833, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1741377549, 'localtime': '2025-03-07 20:59'}, 'current': {'last_updated_epoch': 1741376700, 'last_updated': '2025-03-07 20:45', 'temp_c': 10.3, 'temp_f': 50.5, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 9.8, 'wind_kph': 15.8, 'wind

In [5]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the time where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the time where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_p2XNOIO55ppdPwITspqJqF6Q)
 Call ID: call_p2XNOIO55ppdPwITspqJqF6Q
  Args:
    query: current time in Madrid, Spain
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.timeanddate.com/worldclock/spain/madrid", "content": "Current Local Time in Madrid, Madrid, Spain Time Zone News Calendar & Holiday News Time Zones Time Zones Home Time Zone Converter Time Zone Map Time Difference Time Zone News Calendar & Holiday News Moon Calculator Calculators Home   Time Zones   World Clock   Spain   Madrid Current Local Time in Madrid, Madrid, Spain Time Zone Sunrise & Sunset Thursday, February 13, 2025 Time Zone More Sun & Moon in Madrid Time Zone Converter for Madrid Event Time An